### Import Required Libraries and Set Up Environment Variables

In [228]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')


In [231]:
# Check if the API key was retrieved successfully
if NASA_API_KEY is None:
    print("API key not found. Please check your .env file.")
else:
    print("API key retrieved successfully.")
    print(f"Type of API key: {type(NASA_API_KEY)}")

API key retrieved successfully.
Type of API key: <class 'str'>


### CME Data

In [232]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [236]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)

In [237]:
# Convert the response variable to json and store it as a variable named cme_json
if cme_response.status_code == 200:
    cme_json = cme_response.json()
    print("CME data converted to JSON successfully.")
else:
    print(f"Failed to retrieve CME data. Status code: {cme_response.status_code}")

CME data converted to JSON successfully.


In [238]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
if cme_json and isinstance(cme_json, list) and len(cme_json) > 0:
    first_cme = cme_json[0]  # Get the first CME entry
    print(json.dumps(first_cme, indent=4))  # Print it in a formatted way
else:
    print("No CME data available.")

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [239]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

# Display the first few rows of the DataFrame to verify
cme_df.head()
#cme_df.info()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [242]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])

cme_df.head()
#cme_df.info()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [243]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index in cme_df.index:
    # Get the corresponding value from row i in 'activityID'
    activity_id = cme_df.loc[index, 'activityID']
    # Get the corresponding value from row i in 'startTime' 
    start_time = cme_df.loc[index, 'startTime']       
    # Get the list of dictionaries from row i in 'linkedEvents'
    linkedEvents = cme_df.loc[index, 'linkedEvents']
    

    # Iterate over each dictionary in the list
    for item in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
            'activityID': activity_id,
            'startTime': start_time,
            'linkedEvent': item
        })    
# Create a new DataFrame from the expanded row
expanded_cme_df = pd.DataFrame(expanded_rows)


expanded_cme_df.head()
#expanded_cme_df.info()

,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [244]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

def extract_activityID_from_dict(input_dict):
    try:
        # Attempt to return the 'activityID' from the input dictionary
        return input_dict['activityID']
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        return f"Error: Invalid input. Expected a dictionary with 'activityID', but received: {input_dict}"

# Test the function using one row from linkedEvents
example_linked_event = expanded_cme_df.loc[0, 'linkedEvent']  # Get the first linked event from the first row
activity_id = extract_activityID_from_dict(example_linked_event)  # Pass the dictionary directly
print(activity_id)  # Prints the extracted activityID or the error message if there was an error



2013-05-04T04:52:00-IPS-001


In [245]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))

expanded_cme_df.head()
#expanded_df.info()

,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [248]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df = expanded_cme_df.dropna(subset=['GST_ActivityID'])

expanded_cme_df.head()
#expanded_cme_df.info()

,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [249]:
# print out the datatype of each column in this DataFrame:
expanded_cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvent     1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB


In [250]:
# Convert the 'GST_ActivityID' column to string format 
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format  
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'], utc=True)

# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df = expanded_cme_df.rename(columns={'startTime': 'startTime_CME','activityID': 'cmeID'})

# Drop linkedEvents
expanded_cme_df = expanded_cme_df.drop(columns=['linkedEvent'])

# Verify that all steps were executed correctly
expanded_cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 40.3+ KB


In [253]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_cme_df = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST')]

expanded_cme_df.head()
#expanded_cme_df.info()


,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [259]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"




In [260]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

print(gst_response)

<Response [200]>


In [261]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data


if gst_response.status_code == 200:
    gst_json = gst_response.json()
    print("GST data converted to JSON successfully.")
else:
    print(f"Failed to retrieve GST data. Status code: {gst_response.status_code}")

GST data converted to JSON successfully.


In [262]:
# Convert gst_json to a Pandas DataFrame  
gst_df_initial = pd.DataFrame(gst_json)

gst_df_initial.head()

# Extract the activityID from linkedEvents
gst_df_initial['activityID'] = gst_df_initial['linkedEvents'].apply(lambda x: x[0]['activityID'] if isinstance(x, list) and len(x) > 0 else None)

# Create a new DataFrame with the relevant columns
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df_initial[['activityID', 'startTime', 'linkedEvents']]

# Display the first few rows of the final DataFrame
gst_df.head()


,activityID,startTime,linkedEvents
0,2013-05-31T15:45:00-HSS-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-02T20:24:00-CME-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,None,2013-06-29T03:00Z,None
3,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-04T23:12:00-CME-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [263]:
gst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activityID    105 non-null    object
 1   startTime     117 non-null    object
 2   linkedEvents  105 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [266]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df[gst_df['linkedEvents'].notna()]

gst_df.head()
#gst_df.info()


,activityID,startTime,linkedEvents
0,2013-05-31T15:45:00-HSS-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-02T20:24:00-CME-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-04T23:12:00-CME-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-16T14:15:00-CME-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df_exploded = gst_df.explode('linkedEvents', ignore_index=True)

gst_df_exploded = gst_df_exploded.dropna(subset=['linkedEvents'])

gst_df_exploded.reset_index(drop=True, inplace=True)

gst_df_exploded.head()


,activityID,startTime,linkedEvents
0,2013-05-31T15:45:00-HSS-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-02T20:24:00-CME-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [270]:
gst_df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activityID    205 non-null    object
 1   startTime     205 non-null    object
 2   linkedEvents  205 non-null    object
dtypes: object(3)
memory usage: 4.9+ KB


In [271]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict['activityID']
    except (ValueError, TypeError) as e:
        return None

gst_df_exploded.loc[:, 'CME_ActivityID'] = gst_df_exploded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_exploded = gst_df_exploded.dropna(subset=['CME_ActivityID'])

gst_df_exploded.head()

,activityID,startTime,linkedEvents,CME_ActivityID
0,2013-05-31T15:45:00-HSS-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-02T20:24:00-CME-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-09-29T22:40:00-CME-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [272]:
gst_df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      205 non-null    object
 1   startTime       205 non-null    object
 2   linkedEvents    205 non-null    object
 3   CME_ActivityID  205 non-null    object
dtypes: object(4)
memory usage: 6.5+ KB


In [273]:
# Convert the 'CME_ActivityID' column to string format 
gst_df_exploded['CME_ActivityID'] = gst_df_exploded['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df_exploded['activityID'] = gst_df_exploded['activityID'].astype(str)
gst_df_exploded.rename(columns={'activityID': 'gstID'}, inplace=True) #I cannot find the gstID, changing the activityID column name to gstID

# Convert startTime to datetime format  
gst_df_exploded['startTime'] = pd.to_datetime(gst_df_exploded['startTime'])

# Rename startTime to startTime_GST 
gst_df_exploded.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df_exploded.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


In [274]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df_exploded = gst_df_exploded[gst_df_exploded['CME_ActivityID'].str.contains('CME')]
gst_df_exploded.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-02T20:24:00-CME-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-09-29T22:40:00-CME-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-04T23:12:00-CME-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-16T14:15:00-CME-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-18T01:25:00-CME-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [286]:
merged_df = gst_df_exploded.merge(expanded_cme_df, left_on='gstID', right_on='cmeID', how='inner')


In [287]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           65 non-null     object             
 1   startTime_GST   65 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  65 non-null     object             
 3   cmeID           65 non-null     object             
 4   startTime_CME   65 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  65 non-null     object             
dtypes: datetime64[ns, UTC](2), object(4)
memory usage: 3.2+ KB


In [280]:
merged_df.tail()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
56,2023-12-14T17:38:00-CME-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-02-28T17:48:00-CME-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-23T01:25:00-CME-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-23T01:25:00-CME-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
60,2024-04-15T06:48:00-CME-001,2024-04-19 18:00:00+00:00,2024-04-15T06:48:00-CME-001,2024-04-15T06:48:00-CME-001,2024-04-15 06:48:00+00:00,2024-04-19T18:00:00-GST-001


In [288]:
# Verify that the new DataFrame has the same number of rows as cme and gst
# Get the number of rows in each DataFrame
num_rows_gst = len(gst_df_exploded)
num_rows_cme = len(expanded_cme_df)
num_rows_merged = len(merged_df)

# Print the results
print(f'Number of rows in gst_df_cleaned: {num_rows_gst}')
print(f'Number of rows in expanded_cme_df: {num_rows_cme}')
print(f'Number of rows in merged_df: {num_rows_merged}')

Number of rows in gst_df_cleaned: 61
Number of rows in expanded_cme_df: 61
Number of rows in merged_df: 65


### Computing the time it takes for a CME to cause a GST

In [289]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

merged_df.head()

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-02T20:24:00-CME-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-09-29T22:40:00-CME-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-04T23:12:00-CME-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-16T14:15:00-CME-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-18T01:25:00-CME-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00


In [290]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
time_diff_description = merged_df['timeDiff'].describe()

print(time_diff_description)

count                           65
mean     3 days 00:21:50.769230769
std      1 days 00:20:38.438204714
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 20:24:00
75%                3 days 17:24:00
max                6 days 03:00:00
Name: timeDiff, dtype: object


### Exporting data in csv format

In [291]:
# Export data to CSV without the index
merged_df.to_csv("merged_data.csv", index=False)